In [ ]:
import math
import pandas as pd
import sys
sys.path.append(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\codigos")
from utils import calcular_cuota

In [ ]:
df = pd.read_excel("../../db/detalle_ofertas_noviembre.xlsx")

In [ ]:
df["desembolso_Potencial"] = pd.to_numeric(df["desembolso_Potencial"], errors="coerce")
df = df[df["desembolso_Potencial"] > 0]

In [ ]:
def plazo_minimo(rci, desembolso, tasa, ingresos):
    try: 
        gamma = 60 - rci
        log1 = math.log((gamma * ingresos) / (gamma * ingresos - desembolso * tasa))
        log2 = math.log(1 + tasa)
        plazo = log1 / log2
        plazo_final =  1.5 * math.ceil(plazo * 100 / 12) * 12
        if plazo_final > 144:
            return None
        elif plazo_final <= 144:
            return plazo_final
    except: 
        return None

In [8]:
df["RCI Interno"] = pd.to_numeric(df["RCI Interno"], errors="coerce")
df["tasa_Potencial"] = pd.to_numeric(df["tasa_Potencial"], errors="coerce")

In [9]:
df.loc[df["RCI Interno"] < 60, "PLAZO MINIMO"] = df.loc[df["RCI Interno"] < 60].apply(lambda x: plazo_minimo(x["RCI Interno"], x["desembolso_Potencial"],
                                                     x["tasa_Potencial"]/1200, x["Ingreso validado"]), axis=1)

df["PLAZO MINIMO"].fillna(0, inplace=True)

In [10]:
df.loc[df["PLAZO MINIMO"] > 0, "NUEVA CUOTA"] = df.loc[df["PLAZO MINIMO"] > 0, "NUEVO RCI"] = df.loc[df["PLAZO MINIMO"] > 0].apply(lambda x: calcular_cuota(
    x["desembolso_Potencial"], x["tasa_Potencial"] / 1200, x["PLAZO MINIMO"]), axis=1)

df["NUEVO RCI"] = df["NUEVA CUOTA"] / df["Ingreso validado"] * 100 + df["RCI Interno"]

In [16]:
cols_interes = ["desembolso_Potencial", "tasa_Potencial", "PLAZO MINIMO", "Ingreso validado","RCI Interno", "NUEVO RCI"]

validos_temp = df[(~df["NUEVO RCI"].isna()) & (df["NUEVO RCI"] <= 60)][cols_interes]
len(validos_temp)

113

In [17]:
validos_temp.to_csv("../../db/Salidas/Ensayo_emilene_V2.csv")

In [15]:
validos_temp.columns

Index(['CODIGO_CLIENTE', 'TOTAL_SALDO', 'CATEGORIA_INTERNO',
       'CATEGORIA_CONTAGIO', 'FLAG_INDICE', 'SEGMENTO', 'PATRONO', 'TIPO_PAGO',
       'QUINTIL_SALDO_BT', 'SALDO_EXT',
       ...
       'Unnamed: 237', 'Unnamed: 238', 'Unnamed: 239', 'Unnamed: 240',
       'Unnamed: 241', 'Unnamed: 242', 'Unnamed: 243', 'PLAZO MINIMO',
       'NUEVA CUOTA', 'NUEVO RCI'],
      dtype='object', length=247)